In [13]:
!pip install pyomo
!pyomo build-extensions

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

**** Building AMPL External function demo library ****
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PkgConfig: /bin/pkg-config (found version "0.29.2") 
-- Configuring done
-- Generating done
-- Build files have been written to: /tmp/tmpp1cn9ek6
[ 50%] Building C object CMakeFiles/asl_external_demo.dir/functions.c.o
[100%] Linking C shared libra

In [14]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [15]:
class Data:
  def __init__(self, filename):
    with open(filename) as file_object:
      self.F, self.E, self.O = [range(int(n)) for n in file_object.readline().split()]
        # F: set of facilities, E: set of equipments, O: set of occupations
      self.K = [int(n) for n in file_object.readline().split()] # K: Built facilities
      self.b = float(file_object.readline()) # b: Budget
      self.c = [] # c: cost of building facilities
      for n in file_object.readline().split():
        if n in self.K:
          self.c.append(0)
        else:
          self.c.append(float(n))
      self.l = [int(n) for n in file_object.readline().split()] # l: lower bound of ICU beds in each facility
      self.u = [int(n) for n in file_object.readline().split()] # u: upper bound of ICU beds in each facility
      self.p = [float(n) for n in file_object.readline().split()] # p: price of each equipment
      self.n = [int(n) for n in file_object.readline().split()] # n: necessary quantity of each equipment for one ICU bed
      self.h = [float(n) for n in file_object.readline().split()] # h: hiring cost of each occupation
      self.r = [float(n) for n in file_object.readline().split()] # r: necessary rate of each occupation per ICU bed
      self.a = [] # a: availability of each equipment in each facility
      for _ in self.F:
        self.a.append([int(n) for n in file_object.readline().split()])
      self.s = [] # s: staff of each occupation in each facility
      for _ in self.F:
        self.s.append([int(n) for n in file_object.readline().split()])

In [16]:
class Model:
  def __init__(self, data):
    model = pyo.ConcreteModel()
    model.F = data.F
    model.E = data.E
    model.O = data.O
    model.K = data.K
    model.x = pyo.Var(model.F, within=pyo.Integers) # x: number of ICU beds in each facility
    model.y = pyo.Var(model.F, within=pyo.Binary) # y: whether each facility is built or not
    model.z = pyo.Var(model.F, model.E, within=pyo.Integers) # z: number of each equipment in each facility
    model.w = pyo.Var(model.F, model.O, within=pyo.Integers) # w: number of each occupation in each facility
    # Objective function
    model.objective = pyo.Objective(expr=sum(model.x[i] for i in model.F), sense=pyo.maximize)
    # Constraints
    model.budget_constraint = pyo.Constraint(expr=sum(data.c[i] * model.y[i] + sum(data.p[j] * model.z[i, j]
      for j in model.E) + sum(data.h[k] * model.w[i, k] for k in model.O) for i in model.F) <= data.b)
    model.equipment_constraint = pyo.ConstraintList()
    for i in model.F:
      for j in model.E:
        model.equipment_constraint.add(data.a[i][j] + model.z[i, j] >= data.n[j] * model.x[i])
    model.staff_constraint = pyo.ConstraintList()
    for i in model.F:
      for k in model.O:
        model.staff_constraint.add(data.s[i][k] + model.w[i, k] >= data.r[k] * model.x[i])
    model.bed_limit_constraint = pyo.ConstraintList()
    for i in model.F:
      model.bed_limit_constraint.add(data.l[i] * model.y[i] <= model.x[i])
      model.bed_limit_constraint.add(model.x[i] <= data.u[i])
    model.y_fix_constraint = pyo.ConstraintList()
    for i in model.K:
      model.y_fix_constraint.add(model.y[i] == 1)
    model.y_dependent_constraint = pyo.ConstraintList()
    for i in model.F:
      model.y_dependent_constraint.add(model.x[i] / data.u[i] <= model.y[i])
      model.y_dependent_constraint.add(model.y[i] <= model.x[i])
    opt = pyo.SolverFactory('appsi_highs')
    results = opt.solve(model, tee=True)
    results.write()

In [17]:
model = Model(Data('../instances/mock.txt'))

Running HiGHS 1.5.3: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
37 rows, 33 cols, 98 nonzeros
29 rows, 27 cols, 81 nonzeros
23 rows, 21 cols, 63 nonzeros
23 rows, 21 cols, 60 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   23 rows
   21 cols (3 binary, 18 integer, 0 implied int., 0 continuous)
   60 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   75              -inf                 inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   54.87012987     52                 5.52%        0      0      0        18     0.0s
 C       0       0         0   0.00%   54              53                 1.89%       82      8      0        30     0.0s
 H       0       0    